In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import network
import dataloader as dl

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [3]:
Tokenizer = dl.Tokenizer()

In [4]:
train_dataset = dl.SpamData('train.csv', Tokenizer, 180)
Tokenizer = train_dataset.get()
test_dataset = dl.SpamData('test.csv', Tokenizer, 180)
Tokenizer = test_dataset.get()

The word is already in the dictionary. Stupid monkey >:(
The word is already in the dictionary. Stupid monkey >:(
The word is already in the dictionary. Stupid monkey >:(


In [5]:
for i in range(len(train_dataset)):
    print(train_dataset[i][1])

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [6]:
train_data = DataLoader(train_dataset, 64, True)
test_data = DataLoader(train_dataset, 64)

In [7]:
model = network.Network().to(device)

In [8]:
loss_fn = nn.NLLLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=1e-3)

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    model.train()
    losses = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        # y = torch.nn.functional.one_hot(y, num_classes=2)
        loss = loss_fn(pred, y)
        losses += loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch completed. Avg loss: {losses/len(dataloader):>8f}")     

In [10]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")

In [11]:
def using(model, data):
    model.eval()
    with torch.no_grad():
        data.to(device)
        pred = model(data)
        return Tokenizer.Idx2Word(pred.argmax().item())

In [12]:
epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch+1} is started!")
    train(train_data, model, loss_fn, optimizer)
    test(test_data, model, loss_fn)

Epoch 1 is started!
Epoch completed. Avg loss: 4.634102
Test Error: 
 Accuracy: 98.7%, Avg loss: 0.186587
Epoch 2 is started!
Epoch completed. Avg loss: 0.100905
Test Error: 
 Accuracy: 99.7%, Avg loss: 0.030265
Epoch 3 is started!
Epoch completed. Avg loss: 0.019968
Test Error: 
 Accuracy: 99.9%, Avg loss: 0.009015
Epoch 4 is started!
Epoch completed. Avg loss: 0.007580
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.005203
Epoch 5 is started!
Epoch completed. Avg loss: 0.005076
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.003955
Epoch 6 is started!
Epoch completed. Avg loss: 0.003988
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.003641
Epoch 7 is started!
Epoch completed. Avg loss: 0.003623
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.003602
Epoch 8 is started!
Epoch completed. Avg loss: 0.003586
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.003568
Epoch 9 is started!
Epoch completed. Avg loss: 0.003553
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.003537
Epoch 10 is started!
Epoch completed. Av

In [13]:
import re
text = [0] * 180
pattern = r'[^a-zA-Z0-9\s]'
text_tmp = "Hi Akiro, it’s been a while! Let’s grab coffee next week and catch up. Let me know your availability. Looking forward to it! Best, FordMerding"
text_tmp = re.sub(pattern, '', text_tmp)
text_tmp = text_tmp.split()
for i in range(len(text_tmp)):
    text[i] = Tokenizer.Word2Idx(text_tmp[i])
text = torch.tensor(text, dtype=torch.float)    
using(model, text)
# print(train_dataset[0][0])

'spam'

In [14]:
cnt = 0
for text, label in train_dataset:
    if label == 1:
        cnt+=1
print(cnt, len(train_dataset))

5572 5572
